In [2]:
!pip install torch_geometric torch torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-2.2.0+cu118.html


Looking in links: https://data.pyg.org/whl/torch-2.2.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 76.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 

In [15]:
import networkx as nx
from torch_geometric.utils import from_networkx
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
import community.community_louvain as community_louvain
from networkx.algorithms.community import label_propagation_communities, girvan_newman, asyn_lpa_communities
import itertools
import ipywidgets as widgets
from IPython.display import display, clear_output


In [8]:
!pip install python-louvain


In [18]:
# Upload and read the GraphML file
from google.colab import files
uploaded = files.upload()

# Load the first uploaded file
import io
filename = list(uploaded.keys())[0]
G = nx.read_graphml(io.BytesIO(uploaded[filename]))

# Ensure nodes have string type
G = nx.convert_node_labels_to_integers(G, label_attribute='original_id')

Saving cust_item_network.graphml to cust_item_network.graphml


In [19]:

# Apply Louvain community detection
partition = community_louvain.best_partition(G)
louvain_labels = list(partition.values())

# One-hot features
node_list = list(G.nodes())
for i, node in enumerate(node_list):
    G.nodes[node]['x'] = [int(i == j) for j in range(len(node_list))]
    G.nodes[node]['y'] = louvain_labels[i]

# Convert to PyG data format
from torch_geometric.utils import from_networkx
data = from_networkx(G)
data.x = torch.tensor([G.nodes[n]['x'] for n in node_list], dtype=torch.float)
data.y = torch.tensor([G.nodes[n]['y'] for n in node_list], dtype=torch.long)
data.train_mask = torch.ones(data.num_nodes, dtype=torch.bool)


In [20]:

class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.conv1 = GCNConv(data.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, len(set(louvain_labels)))

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.3, training=self.training)
        x = self.conv2(x, edge_index)
        return x

# Train GCN
model = GCN(hidden_channels=8)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

for epoch in range(101):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.cross_entropy(out, data.y)
    loss.backward()
    optimizer.step()

# GCN predictions
model.eval()
pred = model(data.x, data.edge_index).argmax(dim=1).numpy()


In [21]:

# Interactive toggle for node labels
toggle = widgets.Checkbox(
    value=True,
    description='Show Node Labels',
    disabled=False
)

def plot_graphs(show_labels):
    pos = nx.spring_layout(G, seed=42)
    fig, axes = plt.subplots(1, 2, figsize=(16, 7))

    nx.draw(G, pos, node_color=louvain_labels, cmap='tab10', with_labels=show_labels,
            ax=axes[0], node_size=500, font_weight='bold')
    axes[0].set_title("Louvain Communities")

    nx.draw(G, pos, node_color=pred.tolist(), cmap='tab10', with_labels=show_labels,
            ax=axes[1], node_size=500, font_weight='bold')
    axes[1].set_title("GCN-Predicted Communities")

    plt.tight_layout()
    plt.show()

widgets.interact(plot_graphs, show_labels=toggle)


interactive(children=(Checkbox(value=True, description='Show Node Labels'), Output()), _dom_classes=('widget-i…

<function __main__.plot_graphs(show_labels)>